# Funnel Decomposition Analysis - Demo

This notebook demonstrates the funnel decomposition analysis workflow, showing how to:
1. Load booking funnel data
2. Calculate hierarchical decomposition effects
3. Visualize results with waterfall charts
4. View detailed breakdowns by dimension

In [ ]:
import sys
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Add src to path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

import hier_decomposition_calculator
import visualization_engine

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 12)
plt.rcParams['figure.dpi'] = 100

## 1. Load Mock Data

The dataset contains 24 months of booking funnel data with:
- **Dimensions**: FICO bands (High/Med/Low/Null), offer competition tier, product line
- **Funnel metrics**: Applications, approval rates, booking rates
- **Segments**: 24 segments per month (4 FICO bands × 3 offer tiers × 2 product lines)

In [ ]:
# Load data
data_path = Path.cwd().parent / 'data' / 'funnel_data_mock_v2.csv'
df = pd.read_csv(data_path)
df['month_begin_date'] = pd.to_datetime(df['month_begin_date'])

print(f"Loaded {len(df)} rows of data")
print(f"\nDate range: {df['month_begin_date'].min().date()} to {df['month_begin_date'].max().date()}")
print(f"Unique months: {df['month_begin_date'].nunique()}")
print(f"Segments per month: {len(df) // df['month_begin_date'].nunique()}")

# Show sample data
print("\nSample data:")
df.head(10)

## 2. Calculate Decomposition

We'll decompose the booking change from **June 2023 → June 2024** (Year-over-Year) into 6 effects:
1. **Volume Effect**: Change due to total application volume
2. **Mix Effect**: Change due to segment mix (dimension distribution)
3. **Straight Approval Effect**: Change in straight approval rates
4. **Conditional Approval Effect**: Change in conditional approval rates
5. **Straight Booking Effect**: Change in straight booking rates
6. **Conditional Booking Effect**: Change in conditional booking rates

In [ ]:
# Calculate decomposition for June 2023 → June 2024 (Year-over-Year)
results = hier_decomposition_calculator.calculate_decomposition(
    df=df,
    date_a='2023-06-01',
    date_b='2024-06-01',
    lender='ACA'
)

print("Decomposition Results:")
print(f"  Period 1: {results.metadata['date_a']}")
print(f"  Period 2: {results.metadata['date_b']}")
print(f"  Period 1 bookings: {results.metadata['period_1_total_bookings']:,.0f}")
print(f"  Period 2 bookings: {results.metadata['period_2_total_bookings']:,.0f}")
print(f"  Delta bookings: {results.metadata['delta_total_bookings']:+,.0f}")
print(f"\n  Number of segments: {results.metadata['num_segments']}")

## 3. View Summary Table

The summary shows aggregate impact of each effect:

In [ ]:
results.summary

## 4. Create Waterfall Grid

The waterfall grid shows:
- **Top-left**: Overall aggregate waterfall
- **Top-right**: Breakdown by FICO bands (High/Med/Low)
- **Bottom-left**: Breakdown by Offer Comp Tier (solo/multi_best/multi_other)
- **Bottom-right**: Breakdown by Product Line (Used/VMax)

**Color scheme**:
- Green shades = positive contributions
- Red shades = negative contributions
- Gray = Start/End bars

In [ ]:
# Create waterfall grid
fig = visualization_engine.create_waterfall_grid(
    summary=results.summary,
    segment_detail=results.segment_detail,
    lender='ACA'
)

plt.show()

## 5. Detailed Breakdowns by Dimension

For each dimensional waterfall, we can view the exact contribution of each dimension value:

In [ ]:
# Print detailed breakdowns
visualization_engine.print_waterfall_breakdowns(fig)

## 6. Dimension Drilldown Charts

We can create horizontal bar charts showing the impact of each effect broken down by a specific dimension.

### By FICO Band

In [ ]:
# Create FICO band drilldown
fig_fico = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='fico_bands',
    lender='ACA'
)

plt.show()

### By Offer Comp Tier

In [ ]:
# Create Offer Comp Tier drilldown
fig_comp = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='offer_comp_tier',
    lender='ACA'
)

plt.show()

### By Product Line

In [ ]:
# Create Product Line drilldown
fig_prod = visualization_engine.create_dimension_drilldown(
    segment_detail=results.segment_detail,
    dimension='prod_line',
    lender='ACA'
)

plt.show()

## 7. Segment-Level Detail

For detailed analysis, we can examine the segment-level breakdown:

In [ ]:
# Show segment detail (first 10 segments)
results.segment_detail.head(10)

## 8. Export Results (Optional)

Results can be exported using pandas if needed:

In [ ]:
# Export functionality has been removed from the package
# Users can export results using pandas directly:
# results.summary.to_csv('summary.csv', index=False)
# results.segment_detail.to_csv('segment_detail.csv', index=False)

## Summary

This analysis shows:

1. **Overall Impact**: Total booking change of +768 from 4,480 → 5,248
2. **Key Drivers**:
   - Straight Booking rates improved significantly (+1,128)
   - Conditional Booking rates also improved (+335)
   - Volume declined (-532)
   - Approval rates slightly declined (-146 straight, -17 conditional)

3. **By FICO**:
   - High FICO drove most of the booking rate improvements
   - All FICO bands saw volume declines

4. **By Offer Comp**:
   - Solo offers showed strong booking rate improvements
   - Multi offers also contributed positively

5. **By Product**:
   - Used vehicles showed larger improvements in booking rates
   - Both products contributed positively